Assignment:4
1. you have to create one supervisor node.
2. create one router function
3. create three more node
3.1 llm call (llm node)
3.2 RAG (rag node)
3.3 web crawler(fetch the info in realtime from internet)
4. created one more node after this for validation for generated output --> explore the validation part how to do that
5. if validation going to be failed in that case again go to supervioser node and then supervisor node will again decide what needs to be call next
6. once the validation will pass then only generate the final output

submission deadline till 9pm friday

submission instruction:
create your github repo and keep all the assisgnments over there(in that github repo)
i will share one googleform in group after completing your assignment you can share the github link through that google form. 

### Config the model

In [7]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=0.7)
output=model.invoke("hi")
print(output.content)

Hello! How can I assist you today?


### Config the embedding model

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from langchain_openai import OpenAIEmbeddings

# # Initialize OpenAI Embeddings (make sure your OPENAI_API_KEY is set in environment variables)
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# # Example usage
# embedding_vector = embeddings.embed_query("hi")
# print(len(embedding_vector))  # Should print the embedding size, usually 1536 or 1536 tokens for this model


1536


In [2]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
loader=DirectoryLoader("../data2",glob="Internal_HR_Company_Policies.txt",loader_cls=TextLoader)
docs=loader.load()

In [4]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

new_docs=text_splitter.split_documents(documents=docs)
db = Chroma.from_documents(documents=new_docs, embedding=embeddings, persist_directory=None)
retriever = db.as_retriever()

In [5]:
retriever.get_relevant_documents("iran")

/var/folders/yv/yk0qtcss7tv0_tf_b9jfw4r40000gn/T/ipykernel_85689/1101959227.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("iran")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='and operates in 6 countries. It primarily serves clients in the telecom, BFSI, and healthcare sectors.'),
 Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='Document Title: Internal HR & Company Policies – Nexora Technologies Pvt. Ltd.\n\nDocument ID: HRPOL-INT-2025-003\nConfidentiality Level: Internal Use Only\nLast Updated: May 15, 2025'),
 Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='3. Internal Security & Compliance'),
 Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='Nexora Technologies is a global data engineering and AI solutions provider headquartered in Bangalore, India. The company has over 1,200 employees and operates in 6 countries. It primarily serves')]

In [44]:
import operator
from typing import List
from pydantic import BaseModel , Field
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader

In [93]:
from typing import TypedDict, List
class AgentState(TypedDict):
    question: str
    messages: List[str]
    last_route: str
    attempted_routes: List[str]
    validation_passed: bool

In [148]:
# State definition
def initialize_state(question: str):
    return {"question": question, "route": None, "answer": None, "validated": False}

In [102]:
def llm_router(state: AgentState) -> AgentState:
    question = state["question"]

    route_prompt = PromptTemplate(
        template="""You are a routing assistant.
Decide the best method to answer the question.
Options:
- "crawler" for live or real-time info
- "rag" for document-based or internal policy questions
- "llm" for general knowledge

Question: {question}
Answer with only one word: crawler, rag, or llm.""",
        input_variables=["question"]
    )
    
    chain = route_prompt | model | StrOutputParser()
    route = chain.invoke({"question": question}).strip().lower()

    return {**state, "route": route}


In [103]:
def hybrid_router(state: AgentState) -> AgentState:
    routed_state = llm_router(state)
    route = routed_state["route"]
    question = routed_state["question"]

    if route == "rag":
        relevant_docs = retriever.get_relevant_documents(question)
        if not relevant_docs:
            routed_state["route"] = "llm"
        else:
            routed_state["retrieved_docs"] = relevant_docs
    return routed_state


In [108]:
hybrid_router({"question": "What is the use of machine learning"})

{'question': 'What is the use of machine learning', 'route': 'llm'}

In [109]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

def rag_node(state: AgentState) -> AgentState:
    question = state["question"]
    docs = retriever.get_relevant_documents(question)
    context = format_docs(docs)

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""You are an assistant for question-answering tasks. Use the context below to answer the question briefly. 
If you don't know the answer, just say you don't know.

Context:
{context}

Question: {question}

Answer:"""
    )
    rag_chain = prompt | model | StrOutputParser()
    answer = rag_chain.invoke({"context": context, "question": question})

    return {**state, "answer": answer, "route": "rag"}


In [ ]:
def llm_node(state: dict) -> dict:
    question = state["question"]
    answer = model.invoke(question)
    return {**state, "answer": answer, "route": "llm"}

In [110]:
from langchain_community.tools.tavily_search import TavilySearchResults

def crawler_node(state: dict) -> dict:
    question = state["question"]
    search = TavilySearchResults()
    docs = search.invoke({"query": question, "num_results": 3})
    content = "\n".join([doc["content"] for doc in docs])
    prompt = PromptTemplate.from_template("Use this search result to answer: {content}")
    chain = prompt | model | StrOutputParser()
    answer = chain.invoke({"content": content})
    
    return {**state, "answer": answer, "route": "crawler"}


In [ ]:
# def supervisor_node(state: dict) -> dict:
#     routed_state = hybrid_router(state)
#     route = routed_state.get("route")

#     if route == "llm":
#         print("-->Routing to LLM<--")
#         return llm_node(routed_state)
#     elif route == "rag":
#         print("-->Routing to RAG<--")
#         return rag_node(routed_state)
#     elif route == "crawler":
#         print("-->Routing to Crawler<--")
#         return crawler_node(routed_state)
#     else:
#         return {**state, "error": "Unable to route question"}

In [ ]:
# def validate_answer(answer: str) -> bool:
#     # Dummy check: avoid hallucinations like "I'm an AI model..."
#     bad_phrases = ["As an AI", "I don't have", "cannot access"]
#     return not any(p in answer for p in bad_phrases)


In [415]:
def validate_node(state: dict) -> dict:
    answer = state.get("answer", "")
    # Check for unwanted phrases indicating hallucinations
    bad_phrases = ["As an AI", "I don't have", "cannot access"]
    is_valid = not any(p in answer for p in bad_phrases)
    return {**state, "validated": is_valid}


In [ ]:
# def full_conversation(state: dict) -> str:
#     question = state.get("question", "")
#     if not question:
#         raise ValueError("Question must be provided in the state.")
#     print("🧠 Supervisor received question:", question)
#     state = {"question": question}

#     state = supervisor_node(state)
#     state = validate_node(state)

#     if not state.get("validated"):
#         print("❌ Validation failed. Re-routing to supervisor.")
#         state = supervisor_node(state)
#     else:
#         print("✅ Answer passed validation.")

#     return state

In [364]:
def supervisor_node(state: dict) -> dict:
    question = state.get("question", "")
    if not question:
        raise ValueError("Question must be provided in the state.")
    
    print("🧠 Supervisor received question:", question)
    state = {"question": question}

    # Route the question
    routed_state = hybrid_router(state)
    route = routed_state.get("route")

    # Pick node based on route
    if route == "llm":
        print("-->Routing to LLM<--")
        routed_state = llm_node(routed_state)
    elif route == "rag":
        print("-->Routing to RAG<--")
        routed_state = rag_node(routed_state)
    elif route == "crawler":
        print("-->Routing to Crawler<--")
        routed_state = crawler_node(routed_state)
    else:
        return {**state, "error": "Unable to route question"}

    # Validate the answer
    routed_state = validate_node(routed_state)

    # If invalid, retry routing once
    if not routed_state.get("validated"):
        print("❌ Validation failed. Re-routing to supervisor.")
        return supervisor_node({"question": question})  # Recursive retry
    else:
        print("✅ Answer passed validation.")

    return routed_state


In [365]:
result = supervisor_node({"question": "Can you tell me about the HR Policy of AT&T regarding employee benefits?"})
print("\n💬 Final Answer:", result)

🧠 Supervisor received question: Can you tell me about the HR Policy of AT&T regarding employee benefits?


-->Routing to RAG<--
✅ Answer passed validation.

💬 Final Answer: {'question': 'Can you tell me about the HR Policy of AT&T regarding employee benefits?', 'route': 'rag', 'retrieved_docs': [Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='2. HR Policies'), Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='Disclaimer: This document is for internal use only and should not be shared externally. For any clarification, please contact the HR Operations Team.'), Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='Document Title: Internal HR & Company Policies – Nexora Technologies Pvt. Ltd.\n\nDocument ID: HRPOL-INT-2025-003\nConfidentiality Level: Internal Use Only\nLast Updated: May 15, 2025'), Document(metadata={'source': '../data2/Internal_HR_Company_Policies.txt'}, page_content='Platform            | Purpose\n---------------------|------------------------------------\nPeopl

In [366]:
result["answer"]

"I don't know."

In [416]:
from langgraph.graph import StateGraph,END
workflow=StateGraph(AgentState)

In [417]:
# workflow.add_node("full_conversation",full_conversation)

In [418]:
workflow.add_node("Supervisor",supervisor_node)

In [419]:
workflow.add_node("rag",rag_node)

In [420]:
workflow.add_node("llm",llm_node)

In [421]:
workflow.add_node("crawler",crawler_node)

In [422]:
workflow.add_node("validate", validate_node)

In [423]:
workflow.set_entry_point("Supervisor")

In [424]:
# workflow.add_edge("full_conversation", "Supervisor")

In [425]:
workflow.add_conditional_edges("Supervisor", lambda x: x["route"], {
    "llm": "llm",
    "rag": "rag",
    "crawler": "crawler"
})

In [426]:
workflow.add_edge("llm", "validate")
workflow.add_edge("rag", "validate")
workflow.add_edge("crawler", "validate")
workflow.add_conditional_edges("validate", lambda x: "Supervisor" if not x["validated"] else END)

In [427]:
app = workflow.compile()

In [428]:
state = initialize_state("Tell me about HR policy")

In [431]:
state

{'question': 'Tell me about HR policy',
 'route': None,
 'answer': None,
 'validated': False}

In [429]:
result = app.invoke(state)

🧠 Supervisor received question: Tell me about HR policy
-->Routing to RAG<--
✅ Answer passed validation.


In [430]:
result  # This will give you the final answer after validation

{'question': 'Tell me about HR policy'}

In [396]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [140]:
state={"messages":["can you tell me the industrial growth of world's poor economy?"]}

In [141]:
result=app.invoke(state)

Question can you tell me the industrial growth of world's poor economy?
Parsed response: Topic='Not Related' Reasoning="The query asks about the industrial growth of the world's poor economies, which is a global issue not specific to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


In [142]:
result["messages"][-1]

'There\'s no single, easily quantifiable measure of "industrial growth of the world\'s poor economies."  The term "poor economies" itself is imprecise, and the data available is often incomplete or inconsistent across countries.  However, we can discuss some general trends:\n\n**Challenges in Measuring Industrial Growth in Poor Economies:**\n\n* **Definition of "poor":**  Poverty is measured differently (e.g., by GDP per capita, poverty lines, inequality measures).  Different metrics will yield different groups of "poor economies."\n* **Data Collection:** Reliable industrial data is often lacking in many low-income countries. Informal economies (a significant portion of activity in many developing nations) are difficult to track.\n* **Sectoral Variation:** Industrial growth varies drastically within and between countries. Some might see growth in specific sectors (e.g., textiles, agriculture processing) while others struggle.\n* **Global Economic Shocks:** Global events (pandemics, war

In [ ]:
Assignment:4
1. you have to create one supervisor node.
2. create one router function
3. create three more node
3.1 llm call (llm node)
3.2 RAG (rag node)
3.3 web crawler(fetch the info in realtime from internet)
4. created one more node after this for validation for generated output --> explore the validation part how to do that
5. if validation going to be failed in that case again go to supervioser node and then supervisor node will again decide what needs to be call next
6. once the validation will pass then only generate the final output

submission deadline till 9pm friday

submission instruction:
create your github repo and keep all the assisgnments over there(in that github repo)
i will share one googleform in group after completing your assignment you can share the github link through that google form. 

in next class will discuss about the 
1. react agent with multiple tool call
2.  agentic rag